## Import statements

In [ ]:
import pandas as pd
import time

## Set scraper range

In [2]:
month_start = 1
year_start = 2017

month_end = 11
year_end = 2024

## Scraper Code

In [17]:

curr_month  = month_start
curr_year = year_start

finalDF = pd.DataFrame([])

while True:
    print("Scanning month = {} year = {} ".format(curr_month, curr_year))
    url = f'https://www.umpd.umd.edu/stats/incident_logs.cfm?year={curr_year}&month={curr_month}'
    print(url)
    dfs = pd.read_html(url)
    crimeTable = dfs[0]

    crimeTable.drop(columns=['Unnamed: 5'], inplace=True)
    

    crimeTable['location_desc'] = None
    crimeTable['date_occurred'] = None
    crimeTable['time_occurred'] = None
    crimeTable['date_reported'] = None
    crimeTable['time_reported'] = None

    toDrop = []
    for index, row in crimeTable.iterrows():
        if index % 2 == 0:
            location = crimeTable.loc[index + 1]['OCCURRED DATE TIME LOCATION']
            row['location_desc'] = location
            occurDT = str(row['OCCURRED DATE TIME LOCATION']).split(" ")
            row['date_occurred'] = occurDT[0]
            try:
                row['time_occurred'] = occurDT[1]
            except:
                row['time_occurred'] = "{NO TIME}"

            reportDT = str(row['REPORT DATE TIME']).split(" ")
            row['date_reported'] = reportDT[0]
            row['time_reported'] = reportDT[1]
            toDrop.append(index + 1)
            crimeTable.loc[index] = row

    crimeTable.drop(toDrop, inplace=True)

    crimeTable = crimeTable[["UMPD CASE NUMBER", "date_occurred", "time_occurred", "location_desc", "date_reported", "time_reported", "TYPE", "DISPOSITION"]]
    crimeTable.columns = ['case_number', 'date_occurred', 'time_occurred', 'location_desc', 'date_reported', 'time_reported', 'type', 'disposition']

    curr_month += 1

    if curr_month == 13:
        curr_month = 1
        curr_year += 1

    finalDF = pd.concat([finalDF, crimeTable])
    print(f'Appended {curr_month-1} {curr_year} to final dataframe')
    if curr_month > month_end and curr_year == year_end:
        break
    time.sleep(1)  # sleep for 1 second to avoid getting blocked

finalDF.to_csv('UMPD_crime_export_{}_{}_to_{}_{}.csv'.format(month_start, year_start, month_end, year_end))
finalDF

Scanning month = 1 year = 2017 
https://www.umpd.umd.edu/stats/incident_logs.cfm?year=2017&month=1
7600 block of Mowatt Ln
Knox Rd
4300 block of Knox Rd
Championship Ln
7500 block of Calvert Service Ln
8600 block of Boteler Ln
Jull Hall at 4196 Stadium Dr
4000 block of Campus Dr
MFRI, MFRI Office/Classroom Building at 4500 Campus Dr
Chesapeake Building at 4300 Terrapin Trl
4300 block of Knox Rd
3400 block of Tulane Dr
4200 block of Valley Dr
7500 block of Baltimore Ave
8600 block of Boteler Ln
8500 block of Boteler Ln
Lot Y
3900 block of Campus Dr
4300 block of Knox Rd
Baltimore Ave
Hagerstown Hall at 4000 Stadium Dr
7500 block of Yale Ave
8700 block of Boteler Ln
Paint Branch Dr
Computer Science Instructional Center, CSIC at 8169 Paint Branch Dr
Chapel Ln
8100 block of Baltimore Ave
Comcast Center, Xfinity Center at 8500 Paint Branch Dr
3900 block of Campus Dr
Lot 1b
7500 block of Baltimore Ave
North Gate at 4306 Campus Dr
Stamp, Stamp Student Union, Student Union, Union at 3972 Campu

,case_number,date_occurred,time_occurred,location_desc,date_reported,time_reported,type,disposition
0,2017-00000002,01/01/17,00:15,7600 block of Mowatt Ln,01/01/17,00:15,Warrant/Summons Service,Arrest
2,2017-00000034,01/01/17,02:45,Knox Rd,01/01/17,02:50,Disorderly Conduct,Arrest
4,2017-00000036,12/20/16,12:01,4300 block of Knox Rd,01/01/17,04:33,Theft,Suspended
6,2017-00000053,01/01/17,08:51,Championship Ln,01/01/17,08:51,Warrant/Summons Service,Arrest
8,2017-00000103,01/01/17,15:19,7500 block of Calvert Service Ln,01/01/17,15:19,Emergency Petition,CBE
...,...,...,...,...,...,...,...,...
148,2024-00200197,11/14/24,00:56,"Kung Fu Tea, Pho Thom, Ritchie's Columbian at ...",11/14/24,00:56,Assist Other Agency / Disorderly Conduct,CBE
150,2024-00200763,11/14/24,13:50,"Frat 3, Tau Kappa Epsilon, Tau Kappa Epsilon F...",11/14/24,13:57,Injured/Sick Person,CBE
152,2024-00200781,11/14/24,15:00,436 at 4136 Stadium Dr,11/14/24,15:45,Injured/Sick Person,CBE
154,2024-00201124,11/15/24,00:14,3400 block of Tulane Dr,11/15/24,00:14,Injured/Sick Person,CBE
